In [8]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
from elasticsearch import Elasticsearch
import torch

In [9]:
es = Elasticsearch(
    ['http://localhost:9200'],
    http_auth=('kibana_user', 'Tamarinda192!'),
    verify_certs=False
)

/tmp/ipykernel_130508/2600257124.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [10]:
mapping = { 
    'properties': { 
        'embedding': { 
            'type': 'dense_vector', 
            'dims': 768, 
            'index': 'true',
            "similarity": "cosine"
        } 
    } 
} 

In [11]:
es.indices.create(index='tickets', body={'mappings': mapping})

tickets = [
    {
        'description': 'Odoo ERP failing to update sales orders for GCB.',
        'company': 'GCB',
        'config_item': 'odoo',
        'priority': 'P1'
    },
    {
        'description': 'Dragonfly analytics dashboard loading slowly for Liquid Power.',
        'company': 'Liquid_Power',
        'config_item': 'dragonfly',
        'priority': 'P2'
    },
    {
        'description': 'Cisco AMP alerts not generating for GCB.',
        'company': 'GCB',
        'config_item': 'cisco_amp',
        'priority': 'P2'
    },
    {
        'description': 'Defender antivirus needs update on all workstations for Liquid Power.',
        'company': 'Liquid_Power',
        'config_item': 'defender',
        'priority': 'P3'
    },
    {
        'description': 'ERP training session scheduling conflict for project team.',
        'company': 'GCB',
        'config_item': 'task',
        'priority': 'none'
    },
    {
        'description': 'Migration task delayed due to data integrity issues in source ERP system.',
        'company': 'Liquid_Power',
        'config_item': 'task',
        'priority': 'none'
    },
    {
        'description': 'Customization requirements gathering needs rescheduling.',
        'company': 'GCB',
        'config_item': 'task',
        'priority': 'none'
    },
    {
        'description': 'Network switch failure impacting floor 3 connectivity.',
        'company': 'Liquid_Power',
        'config_item': 'network_switch',
        'priority': 'P4'
    },
    {
        'description': 'Periodic latency spikes detected on primary switch.',
        'company': 'GCB',
        'config_item': 'network_switch',
        'priority': 'P4'
    },
    {
        'description': 'Switch firmware update required to patch security vulnerability.',
        'company': 'Liquid_Power',
        'config_item': 'network_switch',
        'priority': 'P4'
    },
    {
        'description': 'User access review task overdue for ERP system roles.',
        'company': 'GCB',
        'config_item': 'task',
        'priority': 'none'
    },
    {
        'description': 'Documentation task incomplete for ERP system configuration settings.',
        'company': 'Liquid_Power',
        'config_item': 'task',
        'priority': 'none'
    },
        {
        'description': 'Orientation schedule setup for new hires in Q2.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Annual bonus calculation discrepancies reported by finance.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Update to the employee handbook required for remote work policy.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'System update needed for new time off request process.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Review and update health benefits package for next fiscal year.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Implement a new performance review tool for all departments.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Plan and execute the annual employee satisfaction survey.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Prepare onboarding kits for new software development team members.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Coordinate the leadership training program for mid-level managers.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Audit and update emergency contact records for all employees.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Develop a new incentive program for sales team performance.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    },
    {
        'description': 'Organize the company-wide team building event for Q3.',
        'company': 'internal',
        'config_item': 'HR',
        'priority': 'none'
    } 
]

In [12]:
# Load the BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Generate embeddings for the tickets using BERT
for ticket in tickets:
    text = ticket['description']
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
        ticket['embedding'] = output.tolist()



In [27]:
# Index the tickets in Elasticsearch
for ticket in tickets:
    es.index(index='tickets', body=ticket)

# Define a sample query vector for a hypothetical support ticket
query = "System crash when trying to generate report in Odoo for GCB"
#"New employee setup needed"
#Network performance issue
#Team building activity planning
#Updating security software
inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
query_vector = output

search = {
    "knn": {
        "field": "embedding",
        "query_vector": query_vector.tolist(),
        "k": 3,
        "num_candidates": 100
    },
    "fields": ["description", "company", "config_item", "priority"]
}

response = es.search(index='tickets', body=search)
for hit in response['hits']['hits']:
    print(f"Ticket: {hit['_source']}")

Ticket: {'description': 'System update needed for new time off request process.', 'company': 'internal', 'config_item': 'HR', 'priority': 'none', 'embedding': [-0.11933054774999619, -0.4207967519760132, 0.2602660655975342, 0.15379741787910461, 0.5869196653366089, 0.008619651198387146, 0.5419707894325256, 0.2952673137187958, -0.2411218285560608, -0.19326265156269073, -0.27815333008766174, -0.17885196208953857, 0.005514932330697775, 0.3568115532398224, -0.44684508442878723, 0.0788271501660347, 0.13718831539154053, 0.031120842322707176, -0.21592897176742554, 0.025947758927941322, 0.12288478016853333, -0.13648101687431335, 0.08809948712587357, 0.06290274113416672, 0.08816913515329361, -0.4333382844924927, -0.427396684885025, -0.07352576404809952, -0.7404325008392334, -0.2240860015153885, -0.1844942718744278, 0.08264037221670151, -0.23004646599292755, -0.08103594183921814, -0.3248913586139679, -0.17506812512874603, 0.14949454367160797, 0.3142765462398529, -0.18114237487316132, 0.23512385785